Name: sharath kasula

Course Number: HDS_5230

Assignment Number: Week_05_Dask_Programming_Assignment

Extract the .csv file from the zipped archive, and load it into a Dask dataframe.

In [28]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
from datetime import datetime

Parallelization is Beneficial

Dataset is large (aggregated global COVID-19 data).
Dask enables parallel file reading and processing, avoiding memory overflow.
Reasoning:
Using Dask (dd.read_csv()) instead of Pandas (pd.read_csv()) allows data to be loaded in chunks, preventing memory issues.
Operations benefiting from Dask:
Dask allows handling large files efficiently.

In [29]:
import dask.dataframe as dd

# Define data types for efficient loading
dtypes = {
    'aggregate': 'object',
    'cases': 'float64',
    'city': 'object',
    'population': 'float64',
    'deaths': 'float64',
    'country': 'object',
    'state': 'object',
    'date': 'object'
}

# Load dataset using Dask with explicit dtypes
df = dd.read_csv("/Users/sharath/Downloads/untitled folder/timeseries.csv", dtype=dtypes, low_memory=False)


Filter Data for U.S. States

 Parallel processing offers advantages.

Filtering operates row by row. Can be executed simultaneously across rows. 
Processing data, in partitions, with Dask allows for filtering capabilities. 
Thinking; 
Efficient filtering, on a distributed dataset can be achieved through Dasks evaluation technique. 

Operations that can take advantage of Dask technology include; 
Parallel filtering reduces computation time.

In [31]:
# Filter for US states only and state-level data
us_df = df[(df['country'] == 'United States') & (df['level'] == 'state')]



Extract data from January 1, 2020 – February 28, 2021.

In [21]:
# Define date range
start_date = '2020-01-01'
end_date = '2021-02-28'
mask = (us_df['date'] >= start_date) & (us_df['date'] <= end_date)
period_df = us_df[mask]


3.2.Compute Per-Capita Mortality Using Dask

Compute Total Deaths & Average Population

In [23]:
import dask.dataframe as dd

# Define explicit data types to prevent dtype mismatches
dtypes = {
    'aggregate': 'object',
    'cases': 'float64',
    'city': 'object',
    'population': 'float64',
    'deaths': 'float64',
    'country': 'object',
    'state': 'object',
    'date': 'object'
}

# Load dataset using Dask with explicit dtypes
df = dd.read_csv("/Users/sharath/Downloads/untitled folder/timeseries.csv", dtype=dtypes, low_memory=False)

# Display first few rows to verify loading
df.head()




,name,level,city,county,state,country,population,lat,long,url,...,recovered,active,tested,hospitalized,hospitalized_current,discharged,icu,icu_current,growthFactor,date
0,"Antwerp, Flanders, Belgium",county,<NA>,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-22
1,"Antwerp, Flanders, Belgium",county,<NA>,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2020-01-23
2,"Antwerp, Flanders, Belgium",county,<NA>,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2020-01-24
3,"Antwerp, Flanders, Belgium",county,<NA>,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2020-01-25
4,"Antwerp, Flanders, Belgium",county,<NA>,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2020-01-26


Per-Capita Mortality=
TotalDeaths/Avg Population


 "Parallel processing offers advantages."

Involves combining rows together by performing calculations like adding them up or finding the average.
With Dask you can aggregate data in pieces of storing everything in memory all at once. 
Thinking through; 
Aggregations, like sum() and mean() can be spread out across sections in Dask to speed up processing, for datasets. 
Industries that can leverage Dask technology; 
Using .compute() at the end ensures efficient execution.

In [24]:
# Convert date column to datetime format
df['date'] = dd.to_datetime(df['date'])

# Filter only U.S. state-level data
us_df = df[(df['country'] == 'United States') & (df['aggregate'] == 'state')]

# Define date range for filtering
start_date = '2020-01-01'
end_date = '2021-02-28'

# Apply date range filter
period_df = us_df[(us_df['date'] >= start_date) & (us_df['date'] <= end_date)]

deaths_by_state = period_df.groupby('state')['deaths'].agg(['min', 'max']).compute()
total_deaths = deaths_by_state['max'] - deaths_by_state['min']

avg_population = period_df.groupby('state')['population'].mean().compute()

state_metrics = pd.DataFrame({'total_deaths': total_deaths, 'avg_population': avg_population})
state_metrics['per_capita_mortality'] = state_metrics['total_deaths'] / state_metrics['avg_population']


Compute Monthly Case Fatality Rate (CFR)

 "Utilizing computing offers advantages."

Each calculation of the CFR, for a month is done separately without any connection, to future calculations. 

Thinking through; 
Than calculating the CFR month by month sequentially Dask allows for all months to be computed simultaneously. 
Tasks that can take advantage of Dask; 
Dask speeds up group operations across months.

In [25]:
# Convert date column to datetime
period_df['date'] = dd.to_datetime(period_df['date'])

# Extract year-month for grouping
period_df['year_month'] = period_df['date'].dt.to_period('M')

# Compute total cases and deaths per state per month
cfr_df = period_df.groupby(['state', 'year_month']).agg({
    'deaths': 'sum',
    'cases': 'sum'
}).compute()

# Compute CFR
cfr_df['CFR'] = (cfr_df['deaths'] / cfr_df['cases']).fillna(0)


# Reshape data to create a 50x14 matrix
cfr_matrix = cfr_df.pivot_table(values='CFR', index='state', columns='year_month')

# Display CFR matrix
print(cfr_matrix.head())


year_month      2020-01  2020-02   2020-03   2020-04   2020-05   2020-06  \
state                                                                      
Alabama             NaN      NaN  0.005324  0.029829  0.038967  0.029617   
Alaska              NaN      NaN  0.000000  0.026795  0.024091  0.013322   
American Samoa      NaN      NaN       NaN  0.000000  0.000000  0.000000   
Arizona             0.0      0.0  0.000000  0.029733  0.039077  0.004230   
Arkansas            NaN      NaN  0.009231  0.018736  0.021011  0.015134   

year_month       2020-07  
state                     
Alabama         0.023956  
Alaska          0.014205  
American Samoa  0.000000  
Arizona         0.019471  
Arkansas        0.012545  


3.3. Compute CFR Change Rankings

Parallelization is Beneficial

Computing differences across months is an array operation.
Dask enables optimized .diff() operations.
Reasoning:
Parallel execution of CFR change computations across months reduces execution time.
Operations benefiting from Dask:
Parallelized .diff() improves efficiency.

In [26]:
# Compute month-over-month CFR changes
cfr_changes = cfr_matrix.diff(axis=1).fillna(0)

# Sum absolute month-over-month changes per state
cfr_change_rank = cfr_changes.abs().sum(axis=1).sort_values(ascending=False)

# Convert results to DataFrame
cfr_change_rank_df = pd.DataFrame({'Total CFR Change': cfr_change_rank})

# Display CFR change ranking
print(cfr_change_rank_df.head())


                          Total CFR Change
state                                     
Arizona                           0.089165
New Jersey                        0.083710
Michigan                          0.079918
Northern Mariana Islands          0.079321
Connecticut                       0.075907


Parallelization is NOT Necessary

Sorting operations are best handled by Pandas.
Dask’s overhead is unnecessary for a small dataset (50 states).
Reasoning:
Once the dataset is reduced to only 50 states, Pandas (.rank()) is faster than Dask.
Operations benefiting from Pandas (Single-Threaded Processing):
Pandas handles small-scale ranking efficiently.

Justification for Parallelized Computation

In [27]:
justification = """
Parallelized computation is beneficial for:
- Large datasets: COVID-19 time series data spans multiple states and dates, making parallel processing efficient.
- Aggregation operations: Computing per-state totals, means, and CFR changes benefits from distributed processing.
- CFR computation: Monthly breakdowns involve summation and division operations, ideal for parallel execution.
However, for smaller operations (e.g., ranking CFR changes), single-threaded execution suffices.
"""
print(justification)



Parallelized computation is beneficial for:
- Large datasets: COVID-19 time series data spans multiple states and dates, making parallel processing efficient.
- Aggregation operations: Computing per-state totals, means, and CFR changes benefits from distributed processing.
- CFR computation: Monthly breakdowns involve summation and division operations, ideal for parallel execution.
However, for smaller operations (e.g., ranking CFR changes), single-threaded execution suffices.

